In [22]:
import pandas as pd
import numpy as np
import cv2, os, gc

from glob import glob
##from tqdm import tqdm
from tqdm.notebook import tqdm

import torch

import matplotlib
import matplotlib.pyplot as plt

import os
import gc
import tqdm
import copy
from PIL import Image
import torch
from sklearn.cluster import KMeans
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv('train_withbox_split_clean_encode.csv')
df['img_name'] = df.apply(lambda row: f"img/{row.patient_id}_{row.image_id}.png", axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54659 entries, 0 to 54658
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               54659 non-null  int64 
 1   Unnamed: 0.1             54659 non-null  int64 
 2   site_id                  54659 non-null  int64 
 3   patient_id               54659 non-null  int64 
 4   image_id                 54659 non-null  int64 
 5   laterality               54659 non-null  object
 6   view                     54659 non-null  int64 
 7   age                      54659 non-null  int64 
 8   cancer                   54659 non-null  int64 
 9   biopsy                   54659 non-null  int64 
 10  invasive                 54659 non-null  int64 
 11  BIRADS                   54659 non-null  int64 
 12  implant                  54659 non-null  int64 
 13  density                  54659 non-null  int64 
 14  machine_id               54659 non-nul

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def kmeans_set_zero(img, dsize=(320,512), num_clusters=4):
    # flatten the image
    img_1d = img.reshape(-1,1)
     
    # apply kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(img_1d)
    cluster_ids_x = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_

    smallest_sum = np.inf
    smallest_id = 0
    for cluster_id in range(num_clusters):
        cluster_sum = img_1d[cluster_ids_x == cluster_id].sum() / (100000)
        if cluster_sum < smallest_sum:
            smallest_sum = cluster_sum
            smallest_id = cluster_id

    img_1d[cluster_ids_x == smallest_id] = 0    
    kmeans_img = img_1d.reshape(dsize)
    return kmeans_img

def extraxtor_from_roi_box(d, frame, dsize=(512, 320), num_clusters=4):

    h, w = frame.shape
    org_dsize = (h, w)

    frame = kmeans_set_zero(frame, dsize=org_dsize, num_clusters=num_clusters)
    frame_org = copy.copy(frame)

    thres1 = np.min(frame) + 68
    np.place(frame, frame < thres1, 0)

    thres2 = frame_org.sum() / (h * w)
    vertical_not_zero = [True if frame[:, idx].sum() > thres2 else False for idx in range(w)]
    horizontal_not_zero = [True if frame[idx, :].sum() > thres2 else False for idx in range(h)]

    crop = frame_org[horizontal_not_zero, :]
    crop = crop[:, vertical_not_zero]

    crop = cv2.resize(crop, dsize=dsize, interpolation=cv2.INTER_LINEAR)

    return crop

dsize = (320, 512)
org_images = []
images = []
num = 5

for idx in tqdm.tqdm(range(len(df))):
    d = df.iloc[idx]
    img_file = d.img_name
    frame = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    org_frame = frame.copy()
    crop = extraxtor_from_roi_box(d, frame, dsize=dsize)
    org_images.append(org_frame)
    images.append(crop)

    new_img_name = os.path.join('image/', f"{d.patient_id}_{d.image_id}.png")
    im = Image.fromarray(crop)
    im.save(new_img_name)

fig, axes = plt.subplots(num, num, figsize=(20, 20))
for idx, image in enumerate(org_images):
    i = idx % num
    j = idx // num
    axes[i, j].imshow(image)

plt.subplots_adjust(wspace=0, hspace=.2)
plt.title("Original Images")
plt.show()

fig, axes = plt.subplots(num, num, figsize=(20, 20))
for idx, image in enumerate(images):
    i = idx % num
    j = idx // num
    axes[i, j].imshow(image)

plt.subplots_adjust(wspace=0, hspace=.2)
plt.title("Cropped Images")
plt.show()

del frame, crop, images, image
gc.collect()


  1%|█                                                                          | 729/54659 [16:18<20:36:51,  1.38s/it]

In [14]:
pip show torchvision


Name: torchvision
Version: 0.14.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: c:\anaconda\envs\ss\lib\site-packages
Requires: numpy, pillow, requests, torch, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.
